# 如何在链中使用工具

在本指南中，我们将介绍创建链（Chains）和代理（Agents）以调用[工具（Tools）](/docs/concepts/tools/) 的基本方法。工具几乎可以是任何东西——API、函数、数据库等。通过工具，我们可以将模型的[_能力_](/docs/concepts/capabilities/) 扩展到不仅仅是输出文本/消息。让模型使用工具的关键在于正确地[_提示模型_](/docs/concepts/llms/prompts/) 并[_解析其响应_](/docs/concepts/agent/parse-llm-response/)，以便模型能够选择正确的工具并为它们提供正确的输入。

## 设置

本指南需要安装以下软件包：

In [ ]:
%pip install --upgrade --quiet langchain

如果你想在 [LangSmith](https://docs.smith.langchain.com/) 中跟踪你的运行，请取消注释并设置以下环境变量：

In [ ]:
import getpass
import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

## 创建一个工具

首先，我们需要创建一个可供调用的工具。在本例中，我们将从一个函数创建一个自定义工具。有关创建自定义工具的更多信息，请参阅 [本指南](/docs/how_to/custom_tools)。

In [2]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [3]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [4]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

## 链（Chains）

如果我们知道只需要固定次数地使用某个工具，那么我们可以创建一个链（chain）来完成这个任务。我们来创建一个简单的链，它仅用于计算用户指定的数字。

![tool_chain](../../static/img/tool_chain.svg)

### 工具/函数调用
使用 LLM 来调用工具最可靠的方法之一是使用[工具调用](/docs/concepts/tool_calling/) API（有时也称为函数调用）。这仅适用于明确支持工具调用的模型。您可以在[此处](/docs/integrations/chat/)查看哪些模型支持工具调用，并在[本指南](/docs/how_to/function_calling)中了解更多关于如何使用工具调用的信息。

首先我们将定义模型和工具。我们先从一个单独的工具开始，叫做 `multiply`。

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm"/>

In [5]:
# | echo: false
# | output: false

from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

我们将使用 `bind_tools`，将工具的定义作为每次调用模型的一部分进行传递，以便模型可以在适当的时候调用该工具：

In [6]:
llm_with_tools = llm.bind_tools([multiply])

当模型调用工具时，这将显示在输出的 `AIMessage.tool_calls` 属性中：

In [7]:
msg = llm_with_tools.invoke("whats 5 times forty two")
msg.tool_calls

[{'name': 'multiply',
  'args': {'first_int': 5, 'second_int': 42},
  'id': 'call_8QIg4QVFVAEeC1orWAgB2036',
  'type': 'tool_call'}]

请查看[LangSmith trace 在此](https://smith.langchain.com/public/81ff0cbd-e05b-4720-bf61-2c9807edb708/r)。

### 调用工具

太棒了！我们已经能够生成工具调用。但是如果我们想实际调用这个工具呢？为此，我们需要将生成的工具参数传递给我们的工具。作为一个简单的例子，我们将只提取第一个 `tool_call` 的参数：

In [8]:
from operator import itemgetter

chain = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply
chain.invoke("What's four times 23")

92

在此处查看 [LangSmith trace](https://smith.langchain.com/public/16bbabb9-fc9b-41e5-a33d-487c42df4f85/r)。

## Agent

Chain 非常适合处理我们知道特定工具使用顺序的用户输入。但对于某些用例，我们使用工具的次数取决于输入。在这些情况下，我们希望让模型本身决定使用工具的次数和顺序。[Agent](/docs/concepts/agents/) 正是为此而设计的。

我们将使用一个简单的 LangGraph agent 示例来演示。更多详细信息请参阅[本教程](/docs/tutorials/agents)。

![agent](../../static/img/tool_agent.svg)

In [ ]:
!pip install -qU langgraph

In [9]:
from langgraph.prebuilt import create_react_agent

Agent 也非常棒，因为它们可以轻松使用多个工具。

In [10]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [11]:
# Construct the tool calling agent
agent = create_react_agent(llm, tools)

使用代理，我们可以提出需要任意多次使用我们工具的问题：

In [13]:
# Use the agent

query = (
    "Take 3 to the fifth power and multiply that by the sum of twelve and "
    "three, then square the whole result."
)
input_message = {"role": "user", "content": query}

for step in agent.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result.
================================== Ai Message ==================================
Tool Calls:
  exponentiate (call_EHGS8gnEVNCJQ9rVOk11KCQH)
 Call ID: call_EHGS8gnEVNCJQ9rVOk11KCQH
  Args:
    base: 3
    exponent: 5
  add (call_s2cxOrXEKqI6z7LWbMUG6s8c)
 Call ID: call_s2cxOrXEKqI6z7LWbMUG6s8c
  Args:
    first_int: 12
    second_int: 3
================================= Tool Message =================================
Name: add

15
================================== Ai Message ==================================
Tool Calls:
  multiply (call_25v5JEfDWuKNgmVoGBan0d7J)
 Call ID: call_25v5JEfDWuKNgmVoGBan0d7J
  Args:
    first_int: 243
    second_int: 15
================================= Tool Message =================================
Name: multiply

3645
================================== Ai Message =========

在此查看 [LangSmith trace](https://smith.langchain.com/public/eeeb27a4-a2f8-4f06-a3af-9c983f76146c/r)。